In [1]:
import os, io
from getpass import getpass
from BiConn import BiConn
from Auxs   import hashs
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.exceptions import *
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac, cmac
from cryptography.hazmat.primitives.asymmetric import dh,dsa
from cryptography.hazmat.primitives import serialization,hashes

parameters_dh = dh.generate_parameters(generator=2, key_size=1024,
                                     backend=default_backend())

parameters_dsa = dsa.generate_parameters(key_size=1024,backend=default_backend())

def mmac(key):
     return hmac.HMAC(key,hashes.SHA256(),default_backend())

In [2]:
def Dh(conn):
    pk = parameters_dh.generate_private_key()
    pub = pk.public_key().public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo)

    private_key_dsa = parameters_dsa.generate_private_key()
    pub_dsa = private_key_dsa.public_key().public_bytes( 
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo)
    
    conn.send(pub_dsa)

    signature = private_key_dsa.sign(pub, hashes.SHA256())
   
    peer_pub_dsa =serialization.load_pem_public_key(
        conn.recv(), 
        backend=default_backend())  
    
    conn.send(pub)
    conn.send(signature)
    
    try:
        peer_pub = conn.recv()
        sig = conn.recv()
        peer_pub_dsa.verify(sig, peer_pub, hashes.SHA256())
        print("ok dsa")
    except InvalidSignature:
        print("fail dsa")

    peer_pub_key = serialization.load_pem_public_key(
            peer_pub,
            backend=default_backend())
    shared_key = pk.exchange(peer_pub_key)
    
    my_tag = hashs(bytes(shared_key))
    conn.send(my_tag)
    peer_tag = conn.recv()
    if my_tag == peer_tag:
        print('OK DH')
        return my_tag
    else:
        print('FAIL DH')
        
    pk = None
    pub = None
    peer_pub = None
    peer_pub_key = None
    shared_key = None
    my_tag = None
    peer_tag = None

In [3]:
message_size = 2**10

def Emitter(conn):
    key = Dh(conn)
    
    inputs = io.BytesIO(bytes('7'*message_size,'utf-8'))
    
    inicial = os.urandom(512)
    nonce = random.choices(inicial, k=128)
    iv = np.asarray(nonce)
    
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), 
                        backend=default_backend()).encryptor()
    
    mac = mmac(key)
    
    conn.send(iv)
    buffer = bytearray(32)

    try:     
        while inputs.readinto(buffer): 
            ciphertext = cipher.update(bytes(buffer))
            mac.update(ciphertext)
            conn.send((ciphertext, mac.copy().finalize()))         

        conn.send((cipher.finalize(), mac.finalize()))    # envia a finalização
    except Exception as err:
        print("Erro no emissor: {0}".format(err))

    inputs.close()          # fecha a 'input stream'
    conn.close()            # fecha a conecção
    
    # Eliminar chave
    key = None

In [4]:
def Receiver(conn):
    # Acordo de chaves DH e assinatura DSA
    key = Dh(conn)
    
    # Inicializa um output stream para receber o texto decifrado
    outputs = io.BytesIO()
    
    # Recebe o iv
    iv = conn.recv()
    
    # Cifra
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), 
                        backend=default_backend()).decryptor()
    
    # HMAC
    mac = mmac(key)
    
    # operar a cifra: ler da conecção um bloco, autenticá-lo, decifrá-lo e escrever o resultado no 'stream' de output
    try:
        while True:
            try:
                buffer, tag = conn.recv()
                ciphertext = bytes(buffer)
                mac.update(ciphertext)
                if tag != mac.copy().finalize():
                    raise InvalidSignature("erro no bloco intermédio")
                outputs.write(cipher.update(ciphertext))
                if not buffer:
                    if tag != mac.finalize():
                        raise InvalidSignature("erro na finalização")                
                    outputs.write(cipher.finalize())
                    break
                    
            except InvalidSignature as err:
                raise Exception("autenticação do ciphertext ou metadados: {}".format(err))
        print(outputs.getvalue())     # verificar o resultado
        
    except Exception as err:
        print("Erro no receptor: {0}".format(err))
        
    outputs.close()    # fechar 'stream' de output
    conn.close()       # fechar a conecção
    
    # Eliminar chave
    key = None

In [5]:
BiConn(Emitter, Receiver, timeout=30).auto()